# Исследования рынка общепита в Москве для принятия решения об открытии нового заведения

**Описание проекта**  
Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.

Вам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

**Цель исследования**  
Подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

**Описание данных**  
Файл moscow_places.csv:  
- name — название заведения;
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:  
«Средний счёт: 1000–1500 ₽»;  
«Цена чашки капучино: 130–220 ₽»;  
«Цена бокала пива: 400–600 ₽».  
и так далее;  
- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:  
Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.  
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.  
Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.  
- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:  
Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.  
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.  
Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.  
- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):  
0 — заведение не является сетевым  
1 — заведение является сетевым  
- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.

**Ход исследования**  
Исследование пройдет в 6 этапов:  
1. [Загрузим данные и изучим общую информацию.](#Загрузим-данные-и-изучим-общую-информацию)
2. [Выполним предобработку данных.](#Выполним-предобработку-данных)
3. [Анализ данных.](#Анализ-данных)
4. [Детализируем исследование: открытие кофейни.](#Детализируем-исследование:-открытие-кофейни)
5. [Подготовка презентации.](#Подготовка-презентации)
6. [Общий вывод.](#Общий-вывод)

### Загрузим данные и изучим общую информацию

In [2]:
import pandas as pd
import numpy as np
from numpy import median
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from folium import Map, Marker, Choropleth
from folium.plugins import MarkerCluster

ModuleNotFoundError: No module named 'folium'

In [ ]:
try:
    data = pd.read_csv('/datasets/moscow_places.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')

In [ ]:
data.info()
data.head()

По результатам изучения общей информации можно сделать следующие выводы:

- В данных представлено 8406 заведений.
- В столбцах представлено 14 столбцов с типами данных `object`, `float64`, `int64`.
- Количество строк в столбцах различается. Значит, в данных есть пропущенные значения.

### Выполним предобработку данных
#### Проверим данные на явные и неявные дубликаты

In [ ]:
print('Количество явных дубликатов:', data.duplicated().sum())

In [ ]:
# приведем названия заведений к нижнему регистрку и проверим неявные дубликаты
data['name'] = data['name'].str.lower()
print('Количество неявных дубликатов в столбцах с названием и адресом:', data.duplicated(subset=['name', 'address']).sum())

In [ ]:
# проверка неявных дубликатов в столбце с категорией заведения
data['category'].unique()

In [ ]:
# проверка неявных дубликатов в столбце с административным районом
data['district'].unique()

In [ ]:
# проверка неявных дубликатов в столбце с категорией цен
data['price'].unique()

В данных отсутствуют явные дубликаты. Были найдены 3 неявных дубликата, которые мы оставим как есть.

#### Проверим данные на пропуски

In [ ]:
data.isna().sum()

В данных встречаются пропуски в столбцах с информацией о днях и часах работы, с категориями цен, со средней стоимостью заказа, с оценкой среднего чека, с оценкой одной чашки капучино и с количеством посадочных мест. Так как в столбцах большая доля пропусков, на данный момент оставим как есть.

#### Добавим отдельный столбец с названиями улиц из столбца с адресом

In [ ]:
data['street'] = data['address'].str.split(pat = ", ").str[1]
data.sample(5)

#### Добавим отдельный столбец с обозначением, что заведение работает ежедневно и круглосуточно (24/7)

In [ ]:
data['is_24_7'] = data['hours'].apply(lambda x: True if x == 'ежедневно, круглосуточно' else False)
data.query('hours == "ежедневно, круглосуточно"').head()

**Вывод**

На этапе предобработки данных мы проверили данные на дубликаты и пропуски. В данных отсутствуют явные и неявные дубликаты. В данных встречаются пропуски в столбцах. Так как доля пропусков высокая, на данный момент оставим как есть. Также были добавлен столбец с названиями улиц и столбец с обозначением, что заведение работает ежедневно и круглосуточно (24/7).

### Анализ данных
#### Какие категории заведений представлены в данных? Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построим визуализации.

In [ ]:
data_cat = (data
              .groupby('category', as_index=False)['name']
              .agg('count')
              .sort_values(by='name', ascending=False)
            )
fig = px.bar(data_cat, x='category', y='name', color='category', title='Количество объектов общественного питания по категориям',
            labels={
                     "category": "Категория",
                     "name": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Больше всего объектов общественного питания встречается в категории кафе, затем идут ресторан, кофейня, бар/паб, пиццерия, быстрое питание, столовая и булочная.

#### Исследуем количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построим визуализации. Проанализируем результаты и сделаем выводы.

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.boxplot(x='category', y='seats', data=data)
plt.title('Количество посадочных мест в заведениях по категориям')
plt.xlabel('Категория')
plt.ylabel('Количество посадочных мест');

Так как в данных много выбросов, для визуализации используем медианные значения.

In [ ]:
data_seats = (data
              .groupby('category', as_index=False)['seats']
              .agg('median')
              .round(1)
              .sort_values(by='seats', ascending=False)
             )
fig = px.bar(data_seats, x='category', y='seats', color='category', title='Распределение количества посадочных мест в заведениях по категориям',
            labels={
                     "category": "Категория",
                     "seats": "Количество посадочных мест"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Больше всего посадочных мест – в заведениях в категории ресторан, затем идут бар/паб, кофейня, столовая, быстрое питания, кафе, пиццерия и булочная.

#### Рассмотрим и изобразим соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?

In [ ]:
data_chain = (data
              .pivot_table(index='chain', values='name', aggfunc='count')
              .sort_values(by='chain', ascending=False)
              .reset_index()
             )
fig = go.Figure(data=[go.Pie(labels=['сетевые', 'несетевые'], values=data_chain['name'], textinfo='label+percent')])
fig.update_layout(title_text = 'Cоотношение сетевых и несетевых заведений', showlegend = False)
fig.show() 

Несетевых заведений больше, чем сетевых.

#### Какие категории заведений чаще являются сетевыми? Исследуем данные и ответим на вопрос графиком.

In [ ]:
data_chain_cat = (data
                  .groupby('category', as_index=False)
                  .agg(count=('chain', 'count'), sum=('chain', 'sum'))
                 )
data_chain_cat['ratio'] = round(data_chain_cat['sum'] / data_chain_cat['count'], 3)
data_chain_cat = data_chain_cat.sort_values(by='ratio', ascending=False)
fig = px.bar(data_chain_cat, x='category', y='ratio', color='category', title='Распределение доли сетевых заведений по категориям',
            labels={
                     "category": "Категория",
                     "ratio": "Доля сетевых заведений"
            }
            )
fig.update_layout(showlegend = False)
fig.show()

Доля сетевых заведений больше в категории булочная, затем идут пиццерия, кофейня, быстрое питание, ресторан, кафе, столовая, бар/паб.

#### Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. Построим подходящую для такой информации визуализацию. Знакомы ли нам эти сети? Есть ли какой-то признак, который их объединяет? К какой категории заведений они относятся?

In [ ]:
data_name = (data[data['chain'] == 1]
             .groupby(['name', 'category'], as_index=False)
             .agg(count=('category', 'count'))
             .sort_values(by = 'count', ascending = False)
             .head(15)
            )
fig = px.bar(data_name, x='name', y='count', color='count', title='Топ-15 популярных сетей в Москве',
             labels={
                     "name": "Название заведения",
                     "count": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

In [ ]:
fig = px.bar(data_name, x='name', y='count', color = 'category', title='Топ-15 популярных сетей в Москве по категориям',
             labels={
                     "name": "Название заведения",
                     "count": "Количество заведений",
                     "category": "Категория"
                 }
            )
fig.show()

Большую часть заведений, входящих в топ-15 популярных сетей, составляют кофейни. На первом месте – Шоколадница.

#### Какие административные районы Москвы присутствуют в датасете? Отобразим общее количество заведений и количество заведений каждой категории по районам. Проиллюстрируем эту информацию одним графиком.

In [ ]:
data['district'].unique()

In [ ]:
data_disctrict = (data
                  .groupby(['district', 'category'], as_index=False)
                  .agg(count=('name', 'count'))
                  .sort_values(by='count', ascending=True)
                 )
fig = px.bar(data_disctrict, x='count', y='district', color = 'category', orientation='h', title='Количество заведений каждой категории по районам',
             labels={
                     "district": "Район",
                     "count": "Количество заведений",
                     "category": "Категория"
                 }
            )
fig.show()

В датасете присутствует 9 административных районов: ЦАО, САО, ЗАО, ЮАО, ВАО, СВАО, ЮЗАО, ЮВАО и СЗАО. Больше всего заведений представлено в ЦАО. При этом в ЦАО представлено больше ресторанов, тогда как в других районах первое место занимают кафе.

#### Визуализируем распределение средних рейтингов по категориям заведений. Сильно ли различаются усреднённые рейтинги в разных типах общепита?

In [ ]:
plt.figure(figsize=(15, 8))
data_rating = (data
              .groupby('category', as_index=False)['rating']
              .agg('mean')
              .sort_values(by='rating', ascending=False)
             )
ax = sns.barplot(x='category', y='rating', data=data_rating)
ax.set_ylim(3, 5)
plt.title('Распределение средних рейтингов по категориям заведений')
plt.xlabel('Категория')
plt.ylabel('Средний рейтинг')
plt.grid();

Усреднённые рейтинги не сильно различаются в разных типах общепита. Средний рейтинг во всех категориях находится в диапазоне 4-4.5. На первом месте по среднему рейтингу находится бар/паб, затем идут пиццерия, ресторан, кофейня, булочная, столовая, кафе и быстрое питание.

#### Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете, хранятся в файле admin_level_geomap.geojson.

In [ ]:
data_dist_rat = (data
                 .groupby('district', as_index=False)['rating']
                 .agg('mean')
                 .sort_values(by='rating', ascending=False)
                )

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_dist_rat,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний рейтинг заведений',
).add_to(m)

# выводим карту
m

Самый высокий рейтинг – у заведений в районе ЦАО. Самый низкий - у заведений в районе ЮВАО.

#### Отобразим все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

In [ ]:
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)
# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m

#### Найдем топ-15 улиц по количеству заведений. Построим график распределения количества заведений и их категорий по этим улицам. Проиллюстрируем эту информацию одним графиком.

In [ ]:
data_street = data['street'].value_counts().head(15)
data_street

Первое место в топ-15 улиц по количеству заведений занимает проспект Мира (184 заведения). На последнем месте в списке – Пятницкая улица (48 заведений).

In [ ]:
data_str_cat = (data[data['street'].isin(data_street.index)]
                .groupby(['street', 'category'], as_index=False)
                .agg(count=('name', 'count'))
                .sort_values('count', ascending = False)
               )
fig = px.bar(data_str_cat, x='count', y='street', color = 'category', orientation='h', title='Распределение количества заведений и их категорий по улицам, входящим в топ-15',
             labels={
                     "street": "Название улицы",
                     "count": "Количество заведений"
                 }
            )
fig.show()

В топ-15 улиц по количеству заведений большую часть категорий составляют кафе, ресторан и кофейня.

#### Найдем улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?

In [ ]:
data_one = data.groupby(['street'], as_index=False)['name'].nunique()
data_one = data_one[data_one['name'] == 1]
print('Всего в датасете', len(data_one), 'улиц, на которых находится только один объект общепита.')

In [ ]:
data_district_one = (data[data['street'].isin(list(data_one['street']))]
                .groupby(['district'], as_index=False)
                .agg(count=('street', 'count'))
                .sort_values('count', ascending = False)
               )
fig = px.bar(data_district_one, x='district', y='count', color='district', title='Распределение количества улиц, на которых только один объект общепита, по районам',
                labels={
                     "district": "Район",
                     "count": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Больше всего улиц с одним объектом общепита находится в ЦАО, меньше всего – в ЮЗАО.

In [ ]:
data_category_one = (data[data['street'].isin(list(data_one['street']))]
                .groupby(['category'], as_index=False)
                .agg(count=('street', 'count'))
                .sort_values('count', ascending = False)
               )
fig = px.bar(data_category_one, x='category', y='count', color='category', title='Количество улиц, на которых только один объект общепита, по категориям',
             labels={
                     "category": "Категория",
                     "count": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Чаще всего на улицах с одним объектом общепита расположены кафе, ресторан, кофейня, реже всего – булочная.

#### Значения средних чеков заведений хранятся в столбце `middle_avg_bill`. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитаем медиану этого столбца для каждого района. Используем это значение в качестве ценового индикатора района. Построим фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируем цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?

In [ ]:
data_avg_bill = (data
              .groupby('district', as_index=False)['middle_avg_bill']
              .agg('median')
              .round(1)
              .sort_values(by='middle_avg_bill', ascending=False)
             )

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_avg_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний чек заведений',
).add_to(m)

# выводим карту
m

Самый высокий средний чек – в ЦАО и ЗАО. Самый низкий – в СВАО, ЮАО и ЮВАО.

#### Проиллюстрируем другие взаимосвязи, которые мы нашли в данных. Исследуем часы работы заведений и их зависимость от расположения и категории заведения. Также исследуем особенности заведений с плохими рейтингами, средние чеки в таких местах и распределение по категориям заведений.

Исследуем часы работы заведений и их зависимость от расположения заведения.

In [ ]:
data_is_24_7_dist = (data.query('is_24_7 == True')
                .groupby(['district', 'is_24_7'], as_index=False)
                .agg(count=('name', 'count'))
                .sort_values('count', ascending = False)
               )
# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_is_24_7_dist,
    columns=['district', 'count'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Количество круглосуточных заведений',
).add_to(m)

# выводим карту
m

Больше всего круглосуточных заведений – в ЦАО, меньше всего – в СЗАО.  

Исследуем часы работы заведений и их зависимость от категории заведения.

In [ ]:
data_is_24_7_cat = (data.query('is_24_7 == True')
                .groupby('category', as_index=False)
                .agg(count=('is_24_7', 'count'))
                .sort_values('count', ascending = False)
               )
fig = px.bar(data_is_24_7_cat, x='category', y='count', color='category', title='Распределение количества круглосуточных заведений по категориям',
             labels={
                     "category": "Категория",
                     "count": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Больше всего круглосуточных заведений – в категории кафе, меньше всего – в категории столовая.  

Исследуем особенности заведений с плохими рейтингами.

In [ ]:
data['rating'].describe()

Верхней границей плохого рейтинга будем считать 4.1.

In [ ]:
data_avg_bill_badrate = (data.query('rating < 4.1')
              .groupby('district', as_index=False)['middle_avg_bill']
              .agg('median')
              .round(1)
              .sort_values(by='middle_avg_bill', ascending=False)
             )

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_avg_bill_badrate,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний чек заведений с плохим рейтингом',
).add_to(m)

# выводим карту
m

У заведений с плохим рейтингом самый высокий средний чек – в ЦАО. Самый низкий средний чек теперь в ЮВАО и СЗАО.

In [ ]:
data_cat_goodrate = (data
                     .query('rating >= 4.1')
                     .groupby('category', as_index=False)['name']
                     .agg('count')
                     .sort_values(by='name', ascending=False)
                    )
fig = px.bar(data_cat_goodrate, x='category', y='name', color='category', title='Количество объектов общественного питания с хорошим рейтингом по категориям',
            labels={
                     "category": "Категория",
                     "name": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

In [ ]:
data_cat_badrate = (data
                     .query('rating < 4.1')
                     .groupby('category', as_index=False)['name']
                     .agg('count')
                     .sort_values(by='name', ascending=False)
                    )
fig = px.bar(data_cat_badrate, x='category', y='name', color='category', title='Количество объектов общественного питания с плохим рейтингом по категориям',
            labels={
                     "category": "Категория",
                     "name": "Количество заведений"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Больше всего объектов общественного питания с плохим рейтингом встречается в категориях кафе, ресторан, быстрое питание, кофейня. По сравнению с заведениями с хорошим рейтингом, которые чаще встречаются в категории ресторан, заведения с плохим рейтингом чаще встречаются в категории кафе.

#### Соберем наблюдения по вопросам выше в один общий вывод.

- Больше всего объектов общественного питания встречается в категориях кафе, ресторан и кофейня.
- Больше всего посадочных мест – в заведениях в категориях ресторан, бар/паб, кофейня.
- Несетевых заведений больше, чем сетевых. Доля сетевых заведений больше в категориях булочная, пиццерия, кофейня.
- Большую часть заведений, входящих в топ-15 популярных сетей, составляют кофейни. На первом месте – Шоколадница.
- Больше всего заведений представлено в ЦАО. При этом в ЦАО представлено больше ресторанов, тогда как в других районах первое место занимают кафе.
- Усреднённые рейтинги не сильно различаются в разных типах общепита. Средний рейтинг во всех категориях находится в диапазоне 4-4.5. На первых местах по среднему рейтингу находятся бар/паб, пиццерия, ресторан. Самый высокий рейтинг – у заведений в районе ЦАО. Самый низкий - у заведений в районе ЮВАО.
- Первое место в топ-15 улиц по количеству заведений занимает проспект Мира (184 заведения). На последнем месте в топе – Пятницкая улица (48 заведений). В топ-15 улиц по количеству заведений большую часть категорий составляют кафе, ресторан и кофейня.
- Больше всего улиц с одним объектом общепита находится в ЦАО, меньше всего – в ЮЗАО. Чаще всего на улицах с одним объектом общепита расположены кафе, рестораны, кофейни.
- Самый высокий средний чек – в ЦАО и ЗАО. Самый низкий – в СВАО, ЮАО и ЮВАО.
- Больше всего круглосуточных заведений – в ЦАО, меньше всего – в СЗАО. Больше всего круглосуточных заведений – в категории кафе, быстрое питание, ресторан.
- У заведений с плохим рейтингом самый высокий средний чек – в ЦАО. Самый низкий средний чек теперь в ЮВАО и СЗАО. По сравнению с заведениями с хорошим рейтингом, которые чаще встречаются в категории ресторан, заведения с плохим рейтингом чаще встречаются в категории кафе.

### Детализируем исследование: открытие кофейни

Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуем определить, осуществима ли мечта клиентов.
#### Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

In [ ]:
print('Всего в датасете', data.query('category == "кофейня"')['name'].count(), 'кофеен.')

In [ ]:
data_dist_coff = (data
                  .query('category == "кофейня"')
                  .groupby('district', as_index=False)
                  .agg(count=('name', 'count'))
                  .sort_values(by = 'count', ascending = False)
                 )

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_dist_coff,
    columns=['district', 'count'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Количество кофеен',
).add_to(m)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)
# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.query('category == "кофейня"').apply(create_clusters, axis=1)

# выводим карту
m

Больше всего кофеен находится в районе ЦАО, на втором месте - САО, там в два раза меньше кофеен. Меньше всего кофеен – в районе СЗАО.

#### Есть ли круглосуточные кофейни?

In [ ]:
print('Количество круглосуточных кофеен:', data.query('category == "кофейня" & is_24_7 == True')['is_24_7'].count())
print('Доля круглосуточных кофеен:', (data.query('category == "кофейня" & is_24_7 == True')['is_24_7'].count()/data.query('category == "кофейня"')['name'].count()*100).round(1), '%')

#### Какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
data_dist_rat_cup = (data
                      .query('category == "кофейня"')
                      .groupby('district', as_index=False)['rating']
                      .agg('mean')
                      .sort_values(by='rating', ascending=False)
                     )
# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_dist_rat_cup,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний рейтинг кофеен',
).add_to(m)

# выводим карту
m

Самый высокий рейтинг – у кофеен в районах ЦАО и СЗАО. Самый низкий - у кофеен в районах СВАО и ЗАО. Средний рейтинг во всех районах находится в диапазоне 4.2-4.3.

#### На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
print('Средняя стоимость чашки капучино во всех заведениях составляет', data.query('category == "кофейня"')['middle_coffee_cup'].mean().round(), 'рублей.')
print('Медианная стоимость чашки капучино во всех кофейнях составляет', data.query('category == "кофейня"')['middle_coffee_cup'].median(), 'рублей.')

In [ ]:
data_dist_cup = (data
            .query('category == "кофейня"')
            .groupby('district', as_index=False)['middle_coffee_cup']
            .agg('median')
            .sort_values(by='middle_coffee_cup', ascending=False)
            )
fig = px.bar(data_dist_cup, x='middle_coffee_cup', y='district', color='district', orientation='h', title='Распределение стоимости чашки капучино по районам',
             labels={
                     "middle_coffee_cup": "Стоимость чашки капучино",
                     "district": "Район"
                 }
            )
fig.update_layout(showlegend = False)
fig.show()

Стоимость чашки капучино зависит от района. В районах ЮЗАО, ЦАО и ЗАО нужно ориентироваться на стоимость 190-200 рублей за чашку капучино. В остальных районах нужно ориентироваться на стоимость от 135 до 165 рублей за чашку капучино.

**Рекомендации**

При открытии кофейни нужно учитывать, что:
- больше всего кофеен находится в районе ЦАО, на втором месте - САО, там в два раза меньше кофеен. Меньше всего кофеен – в районе СЗАО;
- доля круглосуточных кофеен в Москве крайне мала и составляет около 4%;
- самый высокий рейтинг – у кофеен в районах ЦАО и СЗАО. Самый низкий - у кофеен в районах СВАО и ЗАО;
- в районах ЮЗАО, ЦАО и ЗАО нужно ориентироваться на стоимость 190-200 рублей за чашку капучино. В остальных районах нужно ориентироваться на стоимость от 135 до 165 рублей за чашку капучино. Медианная стоимость чашки капучино во всех кофейнях составляет 170 рублей.

Подводя итог, для открытия кофейни рекомендуем рассмотреть район ЦАО и установить стоимость за чашку капучино не менее 170 рублей. При этом не рекомендуем делать кофейню круглосуточной.

### Подготовка презентации

Презентация: https://drive.google.com/file/d/1hGJahso-bL2yw1ukETaO0W0mlJcFs0Sm/view?usp=sharing

### Общий вывод 

**При анализе всех имеющихся данных были сделаны следующие выводы:**  
- Больше всего объектов общественного питания встречается в категориях кафе, ресторан и кофейня.
- Больше всего посадочных мест – в заведениях в категориях ресторан, бар/паб, кофейня.
- Несетевых заведений больше, чем сетевых. Доля сетевых заведений больше в категориях булочная, пиццерия, кофейня.
- Большую часть заведений, входящих в топ-15 популярных сетей, составляют кофейни. На первом месте – Шоколадница.
- Больше всего заведений представлено в ЦАО. При этом в ЦАО представлено больше ресторанов, тогда как в других районах первое место занимают кафе.
- Усреднённые рейтинги не сильно различаются в разных типах общепита. Средний рейтинг во всех категориях находится в диапазоне 4-4.5. На первых местах по среднему рейтингу находятся бар/паб, пиццерия, ресторан. Самый высокий рейтинг – у заведений в районе ЦАО. Самый низкий - у заведений в районе ЮВАО.
- Первое место в топ-15 улиц по количеству заведений занимает проспект Мира (184 заведения). На последнем месте в топе – Пятницкая улица (48 заведений). В топ-15 улиц по количеству заведений большую часть категорий составляют кафе, ресторан и кофейня.
- Больше всего улиц с одним объектом общепита находится в ЦАО, меньше всего – в ЮЗАО. Чаще всего на улицах с одним объектом общепита расположены кафе, рестораны, кофейни.
- Самый высокий средний чек – в ЦАО и ЗАО. Самый низкий – в СВАО, ЮАО и ЮВАО.
- Больше всего круглосуточных заведений – в ЦАО, меньше всего – в СЗАО. Больше всего круглосуточных заведений – в категории кафе, быстрое питание, ресторан.
- У заведений с плохим рейтингом самый высокий средний чек – в ЦАО. Самый низкий средний чек теперь в ЮВАО и СЗАО. По сравнению с заведениями с хорошим рейтингом, которые чаще встречаются в категории ресторан, заведения с плохим рейтингом чаще встречаются в категории кафе.

**При детализированном исследовании кофеен были даны следующие рекомендации:**  
При открытии кофейни нужно учитывать, что:
- больше всего кофеен находится в районе ЦАО, на втором месте - САО, там в два раза меньше кофеен. Меньше всего кофеен – в районе СЗАО;
- доля круглосуточных кофеен в Москве крайне мала и составляет около 4%;
- самый высокий рейтинг – у кофеен в районах ЦАО и СЗАО. Самый низкий - у кофеен в районах СВАО и ЗАО;
- в районах ЮЗАО, ЦАО и ЗАО нужно ориентироваться на стоимость 190-200 рублей за чашку капучино. В остальных районах нужно ориентироваться на стоимость от 135 до 165 рублей за чашку капучино. Медианная стоимость чашки капучино во всех кофейнях составляет 170 рублей.  

Подводя итог, для открытия кофейни рекомендуем рассмотреть район ЦАО и установить стоимость за чашку капучино не менее 170 рублей. При этом не рекомендуем делать кофейню круглосуточной.